In [8]:
import folium
import geopy
from geopy.distance import geodesic
import pandas as pd
import numpy as np
import math
from alextang.algos import makeDistMatrix
from alextang.algos import distance

In [5]:
class Point:
    def __init__(self, latitude, longitude):
        self.latitude = latitude
        self.longitude = longitude

In [6]:
points = []
points.clear()
csv = pd.read_csv("Coordinates.csv")
for index, row in csv.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    points.append(Point(latitude, longitude))

In [7]:
Map = folium.Map(location=[points[0].latitude, points[0].longitude], zoom_start=12)
for point in points:
    folium.Marker(location = [point.latitude, point.longitude], 
                  popup = [point.latitude, point.longitude], 
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)
folium.Marker(location = [points[0].latitude, points[0].longitude], 
                  popup = [points[0].latitude, points[0].longitude], 
                  icon=folium.Icon(color='red', icon='map-marker')).add_to(Map)
Map

In [9]:
distanceMatrix = makeDistMatrix(points)
print(distanceMatrix)

[[0.         1.03459644 1.27244718 ... 3.05099521 4.2867691  6.21685955]
 [1.03459644 0.         1.19610979 ... 2.0793508  4.69335854 6.97037836]
 [1.27244718 1.19610979 0.         ... 2.33195485 3.51367959 5.95099397]
 ...
 [3.05099521 2.0793508  2.33195485 ... 0.         5.28259595 8.07850799]
 [4.2867691  4.69335854 3.51367959 ... 5.28259595 0.         3.06431819]
 [6.21685955 6.97037836 5.95099397 ... 8.07850799 3.06431819 0.        ]]


In [10]:
def nearestPointPath(points, startPoint, distanceMatrix):
    numPoints = len(points)
    visited = [False for _ in range(numPoints)]
    path = [startPoint]
    visited[0] = True
    currentPoint = 0

    while len(path) < numPoints:
        nearestPoint = None
        nearestDistance = math.inf
        for point in range(numPoints):
            if not visited[point]:
                distance = distanceMatrix[currentPoint][point]
                if distance < nearestDistance:
                    nearestPoint = point
                    nearestDistance = distance
        currentPoint = nearestPoint
        path.append(currentPoint)
        visited[currentPoint] = True

    return path

In [11]:
nearestPointPath = nearestPointPath(points, 0, distanceMatrix)
Path = [points[index] for index in nearestPointPath]

In [13]:
for point in points:
    folium.Marker(location = [point.latitude, point.longitude], 
                  popup = [point.latitude, point.longitude], 
                  icon=folium.Icon(color='blue', icon='map-marker')).add_to(Map)
polyline_coordinates = [(point.latitude, point.longitude) for point in Path]
folium.PolyLine(polyline_coordinates, color='green').add_to(Map)
Map

In [26]:
def totalDistance(order):
    totalDistance = 0
    numPoints = len(order)
    for i in range(numPoints - 1):
        point1 = order[i]
        point2 = order[i + 1]
        distance1 = distance(point1, point2)
        totalDistance += distance1
    toBeginning = distance(order[-1], order[0])
    totalDistance += toBeginning
    return totalDistance
totalDistance(Path)

38.98291869510777